In [14]:
# Algorithmic Trader Python 

from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')



In [15]:
# load data and sanitize 

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-01-08'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                start=start_date,
                end=end_date).stack()

df.index.names = ['date','ticker']

df.columns = df.columns.str.lower()



[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2016-01-10 00:00:00 -> 2024-01-08)')


In [16]:
# calculate features and technical indicators

# Garman-Klass Volatility
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

# RSI
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# Bollinger Bands
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

# ATR
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

# MACD
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

# Dollar volume
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df



adj close       close        high         low        open  \
date       ticker                                                               
2016-01-11 A        35.557159   37.939999   38.900002   37.410000   38.709999   
           AAL      39.257919   41.080002   41.200001   39.900002   40.560001   
           AAPL     22.425255   24.632500   24.764999   24.334999   24.742500   
           ABBV     38.137867   53.880001   55.980000   52.830002   55.860001   
           ABT      35.062435   40.730000   40.900002   40.099998   40.770000   
...                       ...         ...         ...         ...         ...   
2024-01-05 YUM     128.339996  128.339996  129.100006  127.440002  128.850006   
           ZBH     119.980003  119.980003  121.300003  119.690002  119.720001   
           ZBRA    252.690002  252.690002  257.160004  252.149994  252.210007   
           ZION     44.049999   44.049999   44.139999   41.770000   41.919998   
           ZTS     194.850006  194.850006  195.940002  193.070007  193.070007   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2016-01-11 A         2818400.0         -0.002025        NaN       NaN   
           AAL      15877500.0          0.000103        NaN       NaN   
           AAPL    198957600.0         -0.003582        NaN       NaN   
           ABBV     10483300.0         -0.054587        NaN       NaN   
           ABT       7839700.0         -0.008591        NaN       NaN   
...                        ...               ...        ...       ...   
2024-01-05 YUM       1408800.0          0.000078  51.979174  4.837772   
           ZBH       1391000.0          0.000087  61.239035  4.765302   
           ZBRA       293500.0          0.000192  52.779791  5.451475   
           ZION      2552400.0          0.000574  60.926847  3.668305   
           ZTS       1088200.0          0.000076  61.047444  5.228573   

                     bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2016-01-11 A            NaN       NaN       NaN       NaN     100.214298  
           AAL          NaN       NaN       NaN       NaN     623.317614  
           AAPL         NaN       NaN       NaN       NaN    4461.674879  
           ABBV         NaN       NaN       NaN       NaN     399.810701  
           ABT          NaN       NaN       NaN       NaN     274.878973  
...                     ...       ...       ...       ...            ...  
2024-01-05 YUM     4.867868  4.897964 -0.073669  0.354146     180.805387  
           ZBH     4.791236  4.817170 -1.062166  0.746261     166.892185  
           ZBRA    5.561812  5.672149 -0.071003  0.815839      74.164516  
           ZION    3.775123  3.881942  0.379873  1.535518     112.433218  
           ZTS     5.274162  5.319751  0.378928  1.504144     212.035777  

[990083 rows x 14 columns]

In [17]:
# Aggregate to monthly level and filter top 150 most liquid stocks for each month

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume   adj close       atr   bb_high    bb_low  \
date       ticker                                                            
2016-02-29 A           91.918044   35.004204 -0.941847  3.620122  3.522492   
           AAL        338.977806   39.288368  0.978469  3.729458  3.547833   
           AAPL      3548.073647   22.125889 -1.024249  3.155194  3.106253   
           ABBV       357.006435   39.060570 -0.483842  3.734669  3.638599   
           ABT        245.865730   33.559799 -0.796850  3.570935  3.483742   
...                          ...         ...       ...       ...       ...   
2024-01-31 ABNB       531.243402  135.979996 -1.049542  5.013877  4.887410   
           CEG        168.186979  116.239998  0.125507  4.804735  4.724048   
           GEHC       180.872613   76.620003 -1.037387  4.410788  4.251088   
           KVUE       510.430410   21.350000 -1.326498  3.124828  3.069709   
           VLTO       109.369166   76.900002 -0.824525  4.453972  4.321392   

                     bb_mid  garman_klass_vol      macd        rsi  
date       ticker                                                   
2016-02-29 A       3.571307         -0.001914 -0.160794  50.730047  
           AAL     3.638645         -0.000319  0.621468  56.743988  
           AAPL    3.130723         -0.003013 -0.309962  50.633158  
           ABBV    3.686634         -0.049102 -0.335900  49.898482  
           ABT     3.527338         -0.009363 -0.345717  48.856094  
...                     ...               ...       ...        ...  
2024-01-31 ABNB    4.950643          0.000144  0.138212  51.351696  
           CEG     4.764392          0.000204 -0.918562  48.774127  
           GEHC    4.330938          0.000270  0.715577  61.023250  
           KVUE    3.097269          0.000142  1.695743  56.680129  
           VLTO    4.387682          0.000158  0.233470  48.650762  

[47264 rows x 9 columns]

In [18]:
# Calculate 5-year rolling average of dollar volume for each stocks before filtering.

data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2017-01-31 AAL      42.760246  1.306814  3.891346  3.789144  3.840245   
           AAPL     28.233547 -1.194920  3.389455  3.332440  3.360948   
           ABBV     45.380974 -1.206865  3.877334  3.814875  3.846105   
           ABT      37.100910 -1.169279  3.643741  3.577130  3.610435   
           ACN     102.222725 -1.024720  4.667883  4.637933  4.652908   
...                       ...       ...       ...       ...       ...   
2024-01-31 WMT     156.710007  0.031785  5.084700  5.016716  5.050708   
           XOM     102.629997  0.225792  4.654719  4.596081  4.625400   
           MRNA    111.120003 -0.346632  4.766961  4.300249  4.533605   
           UBER     57.580002 -0.297406  4.185853  4.073873  4.129863   
           ABNB    135.979996 -1.049542  5.013877  4.887410  4.950643   

                   garman_klass_vol      macd        rsi  
date       ticker                                         
2017-01-31 AAL            -0.000522 -0.169728  42.277602  
           AAPL           -0.001885 -0.108637  67.547668  
           ABBV           -0.029823 -0.326247  49.245611  
           ABT            -0.002947  0.198783  66.811155  
           ACN            -0.004494 -0.452292  41.080224  
...                             ...       ...        ...  
2024-01-31 WMT             0.000104  0.052652  49.679443  
           XOM             0.000066 -0.259984  50.009797  
           MRNA            0.003150  0.786030  70.385338  
           UBER            0.000216  0.461030  49.848406  
           ABNB            0.000144  0.138212  51.351696  

[12665 rows x 8 columns]

In [19]:
# Calculate Monthly Returns for different time horizons as features.

def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
    
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2018-01-31 AAL      52.943863  1.287045  4.089372  3.908840  3.999106   
           AAPL     39.581043 -0.963795  3.781592  3.700759  3.741176   
           ABBV     86.127296  1.477183  4.536019  4.272025  4.404022   
           ABT      56.398464 -0.697199  4.073903  3.956251  4.015077   
           ACN     147.171082 -0.987448  5.015497  4.968583  4.992040   
...                       ...       ...       ...       ...       ...   
2024-01-31 WMT     156.710007  0.031785  5.084700  5.016716  5.050708   
           XOM     102.629997  0.225792  4.654719  4.596081  4.625400   
           MRNA    111.120003 -0.346632  4.766961  4.300249  4.533605   
           UBER     57.580002 -0.297406  4.185853  4.073873  4.129863   
           ABNB    135.979996 -1.049542  5.013877  4.887410  4.950643   

                   garman_klass_vol      macd        rsi  return_1m  \
date       ticker                                                     
2018-01-31 AAL             0.000502  0.612739  53.332150   0.044013   
           AAPL           -0.001000 -0.415133  40.100594  -0.010637   
           ABBV           -0.033621  2.077379  62.305755   0.168700   
           ABT            -0.004175  0.967576  69.274869   0.094398   
           ACN            -0.002706  0.290448  63.135446   0.049709   
...                             ...       ...        ...        ...   
2024-01-31 WMT             0.000104  0.052652  49.679443  -0.005962   
           XOM             0.000066 -0.259984  50.009797   0.026505   
           MRNA            0.003150  0.786030  70.385338   0.117345   
           UBER            0.000216  0.461030  49.848406  -0.064804   
           ABNB            0.000144  0.138212  51.351696  -0.001175   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2018-01-31 AAL      0.037235   0.051541   0.013139   0.028045    0.017961  
           AAPL    -0.012944  -0.001992   0.021212   0.018481    0.028553  
           ABBV     0.079892   0.077893   0.081654   0.063580    0.054846  
           ABT      0.052610   0.048209   0.041483   0.041085    0.035516  
           ACN      0.041987   0.041220   0.039209   0.032852    0.030836  
...                      ...        ...        ...        ...         ...  
2024-01-31 WMT      0.005132  -0.012639  -0.002105   0.005388    0.008421  
           XOM     -0.000535  -0.007237  -0.004436  -0.012864   -0.007387  
           MRNA     0.195875   0.135191  -0.009486  -0.019683   -0.037625  
           UBER     0.010586   0.099837   0.025659   0.071029    0.053152  
           ABNB     0.037450   0.047552  -0.018595   0.014298    0.016975  

[10490 rows x 14 columns]

In [20]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2018-01-31 AAL     0.0557 -0.0318 -0.0129 -0.0076 -0.0096   0.044013
           AAPL    0.0557 -0.0318 -0.0129 -0.0076 -0.0096  -0.010637
           ABBV    0.0557 -0.0318 -0.0129 -0.0076 -0.0096   0.168700
           ABT     0.0557 -0.0318 -0.0129 -0.0076 -0.0096   0.094398
           ACN     0.0557 -0.0318 -0.0129 -0.0076 -0.0096   0.049709
...                   ...     ...     ...     ...     ...        ...
2023-11-30 VRTX    0.0884 -0.0010  0.0165 -0.0389 -0.0099  -0.020160
           VZ      0.0884 -0.0010  0.0165 -0.0389 -0.0099   0.091090
           WFC     0.0884 -0.0010  0.0165 -0.0389 -0.0099   0.131192
           WMT     0.0884 -0.0010  0.0165 -0.0389 -0.0099  -0.047243
           XOM     0.0884 -0.0010  0.0165 -0.0389 -0.0099  -0.020506

[10200 rows x 6 columns]